# Lab 03 — Interoperabilidad: De SQL a FHIR

**BE3006 · Análisis de Datos Biomédicos · Ciclo 1, 2026**

---

En **Lab 1** construiste un EHR relacional.  
En **Lab 2** normalizaste su semántica con ICD-10 y LOINC.

Ahora el paso final: convertir esos datos a **FHIR** (Fast Healthcare Interoperability Resources),
el estándar HL7 que usan hospitales, gobiernos y aplicaciones de salud para intercambiar
información clínica.

---

```
┌──────────────┐     ┌─────────────────────┐    ┌─────────────────┐
│   EHR (SQL)  │───▶│  Terminología       │───▶│  FHIR Bundle    │
│  patients    │     │  ICD-10 / LOINC     │    │  Patient        │
│  admissions  │     │  mapeos             │    │  Encounter      │
│  diagnoses   │     └─────────────────────┘    │  Condition      │
│  labevents   │                                │  Observation    │
└──────────────┘                                └─────────────────┘
```

---

**Objetivos:**

1. Construir el EHR y el servidor de terminología desde el notebook (sin correr scripts manualmente)
2. Ver cómo cada tabla SQL se transforma en un recurso FHIR
3. Entender cómo los recursos FHIR se referencian entre sí
4. Ensamblar un Bundle FHIR listo para intercambio
5. Consultar la especificación oficial de HL7 FHIR


---

## Parte 0 — Setup y conexión

Asegúrate de que Docker esté corriendo: `docker compose up -d`  
Jupyter se conecta a PostgreSQL usando el hostname `db` (nombre del servicio en Docker Compose).


In [ ]:
import json
from pathlib import Path

import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

from fhir.resources.bundle import Bundle
from fhir.resources.condition import Condition
from fhir.resources.encounter import Encounter
from fhir.resources.observation import Observation
from fhir.resources.patient import Patient

# Conexión con encoding correcto (evita que nombres con tildes aparezcan rotos)
DB_URL = 'postgresql://uvg_user:uvg_password@db:5432/health_data?client_encoding=utf8'
engine = create_engine(DB_URL)

with engine.connect() as conn:
    version = conn.execute(text('SELECT version()')).scalar()

print('Conexión exitosa.')
print(version)

### Función `ejecutar_sql()`

En lugar de pedirte que corras scripts manualmente, este notebook los ejecuta directamente.  
Después de cada script, vemos el resultado en tablas para entender qué cambió.


In [ ]:
SQL_DIR = Path('../sql')


def ejecutar_sql(filename, verbose=True):
    """Lee y ejecuta un archivo .sql completo contra la base de datos."""
    sql = (SQL_DIR / filename).read_text(encoding='utf-8')
    conn = psycopg2.connect(
        host='db', dbname='health_data',
        user='uvg_user', password='uvg_password',
        options='-c client_encoding=UTF8'
    )
    try:
        with conn.cursor() as cur:
            cur.execute(sql)  # psycopg2 acepta múltiples statements
        conn.commit()
    finally:
        conn.close()
    if verbose:
        print(f'✓ Ejecutado: {filename}')

### Reset de la base de datos (opcional)

**Primera vez:** no necesitas esto.  
**Si ves datos duplicados o errores al re-ejecutar:** cambia `HACER_RESET = True` y ejecuta esta celda primero.


In [ ]:
HACER_RESET = False  # Cambia a True SOLO si necesitas limpiar la DB

if HACER_RESET:
    _reset = '''
        DROP VIEW IF EXISTS v_diagnoses_coded CASCADE;
        DROP VIEW IF EXISTS v_labitems_coded CASCADE;
        DROP TABLE IF EXISTS labevents CASCADE;
        DROP TABLE IF EXISTS d_labitems CASCADE;
        DROP TABLE IF EXISTS diagnoses CASCADE;
        DROP TABLE IF EXISTS admissions CASCADE;
        DROP TABLE IF EXISTS patients CASCADE;
        DROP TABLE IF EXISTS map_labitem_to_concept CASCADE;
        DROP TABLE IF EXISTS map_diagnosis_text_to_concept CASCADE;
        DROP TABLE IF EXISTS terminology_concepts CASCADE;
        DROP TABLE IF EXISTS terminology_systems CASCADE;
    '''
    _conn = psycopg2.connect(
        host='db', dbname='health_data',
        user='uvg_user', password='uvg_password',
        options='-c client_encoding=UTF8'
    )
    try:
        with _conn.cursor() as cur:
            cur.execute(_reset)
        _conn.commit()
    finally:
        _conn.close()
    print('Base de datos limpia. Continúa con la Parte 1.')
else:
    print('Reset omitido.')

---

## Parte 1 — El modelo EHR

Antes de hablar de FHIR, necesitamos datos clínicos.  
Vamos a construir el modelo relacional del Lab 1, directamente desde este notebook.

El modelo tiene 5 tablas:

```
patients
│  (subject_id PK, external_id, full_name, sex, date_of_birth)
│
└── admissions  (hadm_id PK, subject_id FK, admittime, dischtime, admission_type)
        │
        ├── diagnoses    (diagnosis_id PK, hadm_id FK, diagnosis_text)
        │
        └── labevents    (labevent_id PK, hadm_id FK, labitem_id FK, charttime, value_num)
                │
                └── d_labitems   (labitem_id PK, label, unit)
```

Las **admisiones** son el eje del modelo: diagnósticos y labs cuelgan de ellas,
no directamente del paciente. Esto refleja la realidad clínica: un laboratorio
pertenece a un _encuentro_, no al paciente en abstracto.


In [ ]:
# Crear tablas EHR e insertar datos de muestra
ejecutar_sql('002_ehr_schema.sql')

In [ ]:
print('Tablas EHR creadas:')
display(pd.read_sql('''
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name IN ('patients', 'admissions', 'diagnoses', 'd_labitems', 'labevents')
    ORDER BY table_name;
''', engine))

In [ ]:
print('=== Pacientes ===')
display(pd.read_sql('SELECT * FROM patients ORDER BY subject_id;', engine))

In [ ]:
print('=== Admisiones ===')
display(pd.read_sql('''
    SELECT
        a.hadm_id,
        p.full_name          AS paciente,
        a.admittime::date    AS ingreso,
        a.dischtime::date    AS egreso,
        a.admission_type     AS tipo,
        a.hospital_expire_flag AS fallecio
    FROM admissions a
    JOIN patients p ON p.subject_id = a.subject_id
    ORDER BY a.hadm_id;
''', engine))

In [ ]:
print('=== Diagnósticos (EHR puro) ===')
print('Por ahora la tabla solo tiene diagnosis_text — sin columnas de código.')
print('Las columnas code_system, code, code_display se agregan cuando corramos el script 003.')
display(pd.read_sql('''
    SELECT d.diagnosis_id, d.hadm_id, p.full_name AS paciente,
           d.diagnosis_text
    FROM diagnoses d
    JOIN admissions a ON a.hadm_id = d.hadm_id
    JOIN patients p ON p.subject_id = a.subject_id
    ORDER BY d.diagnosis_id;
''', engine))

In [ ]:
print('=== Catálogo de labs (antes de codificar) ===')
display(pd.read_sql('SELECT * FROM d_labitems ORDER BY labitem_id;', engine))

---

## Parte 2 — El servidor de terminología

Los diagnósticos usan texto libre (`'Sepsis'`, `'Hypertension'`).  
Para que otro sistema los entienda sin ambigüedad, necesitamos mapearlos a **códigos estándar**.

Vamos a construir un mini "terminology server" dentro de la misma base de datos —
son cuatro tablas nuevas, conceptualmente separadas del EHR:

```
SERVIDOR DE TERMINOLOGÍA              EHR
┌───────────────────────┐             ┌──────────────────┐
│  terminology_systems  │             │    diagnoses     │
│   ICD-10, LOINC       │             │  + code_system   │
├───────────────────────┤             │  + code          │◀──┐
│  terminology_concepts │             │  + code_display  │   │
│   A41.9 = Sepsis      │             └──────────────────┘   │ UPDATE
│   2160-0 = Creatinine │   ┌──────────────────────────┐     │
├───────────────────────┤   │ map_diagnosis_text_to_   │─────┘
│  tablas de mapeo      │──▶│ concept                  │
│  (el puente)          │   └──────────────────────────┘
└───────────────────────┘
```


### 2.1 — Crear el esquema de terminología

Este script crea las 4 tablas del terminology server y agrega las columnas
`code_system`, `code`, `code_display` a `diagnoses` y `d_labitems`.


In [ ]:
ejecutar_sql('003_terminology_schema.sql')

In [ ]:
print('Tablas de terminología creadas:')
display(pd.read_sql('''
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name IN (
          'terminology_systems', 'terminology_concepts',
          'map_diagnosis_text_to_concept', 'map_labitem_to_concept'
      )
    ORDER BY table_name;
''', engine))

### 2.2 — Cargar los códigos ICD-10 y LOINC

Este script inserta 12 códigos ICD-10 y 5 códigos LOINC que cubren los
diagnósticos y labs del dataset, más las tablas de mapeo texto → código.


In [ ]:
ejecutar_sql('004_seed_terminology.sql')

In [ ]:
print('=== Sistemas de terminología ===')
display(pd.read_sql('SELECT * FROM terminology_systems;', engine))

In [ ]:
print('=== Conceptos ICD-10 (mini) ===')
display(pd.read_sql('''
    SELECT c.code, c.display, c.concept_type
    FROM terminology_concepts c
    JOIN terminology_systems s ON s.system_id = c.system_id
    WHERE s.system_name = 'ICD-10'
    ORDER BY c.code;
''', engine))

In [ ]:
print('=== Conceptos LOINC (mini) ===')
display(pd.read_sql('''
    SELECT c.code, c.display, c.concept_type
    FROM terminology_concepts c
    JOIN terminology_systems s ON s.system_id = c.system_id
    WHERE s.system_name = 'LOINC'
    ORDER BY c.code;
''', engine))

In [ ]:
print('=== Mapeos: texto local → ICD-10 ===')
display(pd.read_sql('''
    SELECT m.diagnosis_text, s.system_name, c.code, c.display
    FROM map_diagnosis_text_to_concept m
    JOIN terminology_concepts c ON c.concept_id = m.concept_id
    JOIN terminology_systems s ON s.system_id = c.system_id
    ORDER BY m.diagnosis_text;
''', engine))

### 2.3 — Antes / Después: aplicar los mapeos al EHR

Primero el estado **antes**: los diagnósticos tienen texto pero ningún código.


In [ ]:
print('=== ANTES: diagnoses sin códigos ===')
display(pd.read_sql('''
    SELECT diagnosis_id, hadm_id, diagnosis_text, code_system, code, code_display
    FROM diagnoses
    ORDER BY diagnosis_id
    LIMIT 8;
''', engine))

In [ ]:
# Aplicar mapeos: UPDATE en diagnoses y d_labitems usando las tablas de mapeo
ejecutar_sql('005_apply_mappings.sql')

In [ ]:
print('=== DESPUÉS: diagnoses con códigos ICD-10 ===')
display(pd.read_sql('''
    SELECT diagnosis_id, hadm_id, diagnosis_text, code_system, code, code_display
    FROM diagnoses
    ORDER BY diagnosis_id;
''', engine))

In [ ]:
print('=== DESPUÉS: d_labitems con códigos LOINC ===')
display(pd.read_sql('''
    SELECT labitem_id, label, unit, code_system, code, code_display
    FROM d_labitems
    ORDER BY labitem_id;
''', engine))

La base de datos está lista: datos clínicos **y** códigos estándar.  
Ahora podemos construir recursos FHIR interoperables.


---

## Parte 3 — De SQL a FHIR

FHIR define un tipo de recurso para cada concepto clínico.  
La equivalencia con nuestro modelo SQL es:

| SQL                        | FHIR Resource | Descripción                  |
| -------------------------- | ------------- | ---------------------------- |
| `patients`                 | `Patient`     | Identidad demográfica        |
| `admissions`               | `Encounter`   | Encuentro clínico / admisión |
| `diagnoses`                | `Condition`   | Diagnóstico clínico          |
| `labevents` + `d_labitems` | `Observation` | Resultado de laboratorio     |
| (todos juntos)             | `Bundle`      | Paquete de intercambio       |

Para cada recurso seguiremos este patrón:

1. **SQL → DataFrame** (la fuente de datos)
2. **Python → recurso FHIR** (la transformación)
3. **JSON FHIR** (el resultado que se intercambia)
4. **Preguntas** → consultar la especificación oficial


In [ ]:
def mostrar_fhir(recurso, titulo='Recurso FHIR'):
    """Imprime el JSON de un recurso FHIR de forma legible."""
    json_str = recurso.model_dump_json(indent=2, exclude_none=True)
    sep = '=' * 60
    print(sep)
    print(f'  {titulo}')
    print(sep)
    print(json_str)
    print()
    return json_str


def fhir_dt(dt):
    """Convierte un datetime de pandas a string ISO 8601 para FHIR."""
    if dt is None or (hasattr(dt, 'isnull') and pd.isnull(dt)):
        return None
    try:
        if pd.isna(dt):
            return None
    except Exception:
        pass
    return dt.strftime('%Y-%m-%dT%H:%M:%S+00:00')


def sex_to_gender(sex):
    """Convierte el código de sexo local al vocabulario FHIR."""
    return {'M': 'male', 'F': 'female', 'O': 'other'}.get(sex, 'unknown')

In [ ]:
print('=== Admisiones disponibles ===')
display(pd.read_sql('''
    SELECT
        a.hadm_id,
        p.full_name                     AS paciente,
        a.admittime::date               AS ingreso,
        a.dischtime::date               AS egreso,
        a.admission_type                AS tipo,
        a.hospital_expire_flag          AS fallecio,
        COUNT(DISTINCT d.diagnosis_id)  AS n_diagnosticos,
        COUNT(DISTINCT le.labevent_id)  AS n_labs
    FROM admissions a
    JOIN patients p ON p.subject_id = a.subject_id
    LEFT JOIN diagnoses d ON d.hadm_id = a.hadm_id
    LEFT JOIN labevents le ON le.hadm_id = a.hadm_id
    GROUP BY a.hadm_id, p.full_name, a.admittime, a.dischtime,
             a.admission_type, a.hospital_expire_flag
    ORDER BY a.hadm_id;
''', engine))

HADM_ID = 3  # Carlos Pérez — Sepsis + AKI, con muerte hospitalaria
print(f'\nTrabajando con hadm_id = {HADM_ID}')

---

### 3.1 — Patient (Paciente)

**Especificación oficial:** https://hl7.org/fhir/R5/patient.html

El recurso `Patient` representa la identidad demográfica de una persona.

| Campo FHIR            | Fuente SQL               | Descripción                             |
| --------------------- | ------------------------ | --------------------------------------- |
| `id`                  | `patients.subject_id`    | Identificador interno del recurso       |
| `identifier[0].value` | `patients.external_id`   | MRN del hospital                        |
| `name[0].text`        | `patients.full_name`     | Nombre completo                         |
| `gender`              | `patients.sex`           | `male` / `female` / `other` / `unknown` |
| `birthDate`           | `patients.date_of_birth` | Fecha de nacimiento                     |


In [ ]:
# SQL fuente — extraemos paciente + admisión en una sola query
enc = pd.read_sql('''
    SELECT
        a.hadm_id, a.admittime, a.dischtime,
        a.admission_type, a.hospital_expire_flag,
        p.subject_id, p.external_id, p.full_name,
        p.sex, p.date_of_birth, p.date_of_death
    FROM admissions a
    JOIN patients p ON p.subject_id = a.subject_id
    WHERE a.hadm_id = %(hadm_id)s
''', engine, params={'hadm_id': HADM_ID})

print('FUENTE SQL → Patient:')
display(enc[['subject_id', 'external_id', 'full_name', 'sex', 'date_of_birth']])

In [ ]:
row = enc.iloc[0]

patient = Patient.model_validate({
    'resourceType': 'Patient',
    'id': str(row['subject_id']),
    'identifier': [{'system': 'urn:uvg:mrn', 'value': row['external_id']}],
    'name': [{'text': row['full_name']}],
    'gender': sex_to_gender(row['sex']),
    'birthDate': str(row['date_of_birth']) if row['date_of_birth'] is not None else None,
})

mostrar_fhir(patient, f'Patient/{patient.id}')

#### Ejercicio 3.1 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/patient.html** y responde:

**a)** Usamos `name[0].text` para el nombre completo. La spec también define los sub-campos `family` y `given` dentro de `HumanName`. ¿Cuándo preferirías usar `family`/`given` en lugar de `text`? ¿Qué ventaja tiene para la interoperabilidad?

**b)** ¿Qué valor de `gender` usarías para una persona cuyo sexo biológico es desconocido? ¿Y para alguien intersex?

**c)** ¿Por qué el campo `identifier.system` usa una URL (`urn:uvg:mrn`) en lugar del texto `'MRN'`?


**Tus respuestas:**

a) ...

b) ...

c) ...


---

### 3.2 — Encounter (Encuentro Clínico)

**Especificación oficial:** https://hl7.org/fhir/R5/encounter.html

`Encounter` representa una admisión, consulta, o cualquier contacto clínico.
Equivale directamente a la tabla `admissions`.

| Campo FHIR           | Fuente SQL             | Descripción                                   |
| -------------------- | ---------------------- | --------------------------------------------- |
| `id`                 | `admissions.hadm_id`   | Identificador del encuentro                   |
| `status`             | `admissions.dischtime` | `finished` si hay egreso, `in-progress` si no |
| `subject.reference`  | `patients.subject_id`  | Referencia al Patient: `"Patient/2"`          |
| `actualPeriod.start` | `admissions.admittime` | Fecha y hora de ingreso                       |
| `actualPeriod.end`   | `admissions.dischtime` | Fecha y hora de egreso                        |


In [ ]:
print('FUENTE SQL → Encounter:')
display(enc[['hadm_id', 'admittime', 'dischtime', 'admission_type', 'hospital_expire_flag']])

In [ ]:
encounter = Encounter.model_validate({
    'resourceType': 'Encounter',
    'id': str(row['hadm_id']),
    'status': 'finished' if row['dischtime'] is not None else 'in-progress',
    'subject': {'reference': f'Patient/{patient.id}'},
    'actualPeriod': {
        'start': fhir_dt(row['admittime']),
        'end': fhir_dt(row['dischtime'])
    },
})

mostrar_fhir(encounter, f'Encounter/{encounter.id}')

#### Ejercicio 3.2 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/encounter.html** y responde:

**a)** ¿Cuáles son todos los valores posibles del campo `status`? Nombra al menos 4 y explica cuándo se usaría cada uno.

**b)** El Encounter tiene un campo `diagnosis` que podría referenciar Conditions. Sin embargo, nosotros ponemos la referencia al Encounter _desde_ la Condition. ¿Cuál es la diferencia entre ambos enfoques? ¿Cuándo usarías uno u otro?

**c)** ¿Qué campo de Encounter usarías para indicar el tipo de encuentro (emergencia, consulta ambulatoria, hospitalización)? ¿Qué sistema de códigos recomienda la spec?


**Tus respuestas:**

a) ...

b) ...

c) ...


---

### 3.3 — Condition (Diagnóstico)

**Especificación oficial:** https://hl7.org/fhir/R5/condition.html

`Condition` representa un diagnóstico clínico. Gracias al Lab 2, nuestros diagnósticos
tienen **texto libre** Y **código ICD-10**.

FHIR usa `code.coding[]` para los códigos y `code.text` para el texto libre.
Vamos a ver la diferencia entre una Condition **con** código y una **sin** código.

| Campo FHIR              | Fuente SQL                 |
| ----------------------- | -------------------------- |
| `id`                    | `diagnoses.diagnosis_id`   |
| `code.coding[0].system` | `diagnoses.code_system`    |
| `code.coding[0].code`   | `diagnoses.code`           |
| `code.text`             | `diagnoses.diagnosis_text` |
| `subject.reference`     | `"Patient/{subject_id}"`   |
| `encounter.reference`   | `"Encounter/{hadm_id}"`    |


In [ ]:
print('FUENTE SQL → Conditions:')
dx = pd.read_sql('''
    SELECT diagnosis_id, hadm_id, diagnosis_text,
           code_system, code, code_display
    FROM diagnoses
    WHERE hadm_id = %(hadm_id)s
    ORDER BY diagnosis_id;
''', engine, params={'hadm_id': HADM_ID})
display(dx)

In [ ]:
conditions = []

for _, r in dx.iterrows():
    # Si el diagnóstico tiene código → lo incluimos en coding[]
    # Si no tiene código → coding[] queda vacío, solo queda code.text
    coding = []
    if pd.notna(r['code']) and pd.notna(r['code_system']):
        coding.append({
            'system': r['code_system'],
            'code': r['code'],
            'display': r['code_display'] if pd.notna(r['code_display']) else None
        })

    cond = Condition.model_validate({
        'resourceType': 'Condition',
        'id': str(r['diagnosis_id']),
        'clinicalStatus': {
            'coding': [{
                'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical',
                'code': 'active'
            }]
        },
        'code': {'coding': coding, 'text': r['diagnosis_text']},
        'subject': {'reference': f'Patient/{patient.id}'},
        'encounter': {'reference': f'Encounter/{encounter.id}'}
    })
    conditions.append(cond)

print(f'Se construyeron {len(conditions)} Conditions.')

In [ ]:
# Mostrar la primera Condition (con código ICD-10)
mostrar_fhir(conditions[0], f'Condition/{conditions[0].id} — CON código ICD-10')

In [ ]:
# Buscar alguna Condition SIN código
sin_codigo = [c for c in conditions if not c.code.coding]
if sin_codigo:
    mostrar_fhir(sin_codigo[0], f'Condition/{sin_codigo[0].id} — SIN código (solo texto)')
else:
    print('Todos los diagnósticos de esta admisión tienen código ICD-10.')
    print('En el Ejercicio B insertarás uno sin código para ver la diferencia.')

#### Ejercicio 3.3 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/condition.html** y responde:

**a)** ¿Qué valores acepta `clinicalStatus.coding[0].code` además de `active`?  
(Busca el value set `ConditionClinicalStatusCodes` en la página)

**b)** ¿Para qué sirve el campo `verificationStatus`? ¿Cuándo usarías `differential` vs `confirmed` en un contexto de urgencias?

**c)** ¿Qué campo usarías para registrar que la Sepsis comenzó 2 días antes de la admisión?  
(Busca los campos `onset*` en la spec)


**Tus respuestas:**

a) ...

b) ...

c) ...


---

### 3.4 — Observation (Resultado de Laboratorio)

**Especificación oficial:** https://hl7.org/fhir/R5/observation.html

`Observation` es el recurso más versátil de FHIR: cubre laboratorios, signos vitales,
hallazgos clínicos, y más. Cada fila de `labevents` genera una `Observation` individual.

| Campo FHIR                   | Fuente SQL                           |
| ---------------------------- | ------------------------------------ |
| `id`                         | `labevents.labevent_id`              |
| `status`                     | `'final'` (siempre, en este dataset) |
| `category[0].coding[0].code` | `'laboratory'`                       |
| `code.coding[0]`             | LOINC de `d_labitems`                |
| `code.text`                  | `d_labitems.label`                   |
| `effectiveDateTime`          | `labevents.charttime`                |
| `valueQuantity.value`        | `labevents.value_num`                |
| `valueQuantity.unit`         | `d_labitems.unit`                    |


In [ ]:
print('FUENTE SQL → Observations:')
labs = pd.read_sql('''
    SELECT le.labevent_id, le.hadm_id, le.charttime, le.value_num,
           li.label, li.unit,
           li.code_system, li.code, li.code_display
    FROM labevents le
    JOIN d_labitems li ON li.labitem_id = le.labitem_id
    WHERE le.hadm_id = %(hadm_id)s
    ORDER BY le.charttime, le.labevent_id;
''', engine, params={'hadm_id': HADM_ID})
display(labs)

In [ ]:
observations = []

for _, r in labs.iterrows():
    coding = []
    if pd.notna(r['code']) and pd.notna(r['code_system']):
        coding.append({
            'system': r['code_system'],
            'code': r['code'],
            'display': r['code_display'] if pd.notna(r['code_display']) else None
        })

    obs = Observation.model_validate({
        'resourceType': 'Observation',
        'id': str(r['labevent_id']),
        'status': 'final',
        'category': [{'coding': [{
            'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
            'code': 'laboratory'
        }]}],
        'code': {'coding': coding, 'text': r['label']},
        'subject': {'reference': f'Patient/{patient.id}'},
        'encounter': {'reference': f'Encounter/{encounter.id}'},
        'effectiveDateTime': fhir_dt(r['charttime']),
        'valueQuantity': {
            'value': float(r['value_num']),
            'unit': r['unit'],
            'system': 'http://unitsofmeasure.org',
            'code': r['unit']
        }
    })
    observations.append(obs)

print(f'Se construyeron {len(observations)} Observations.')
print('\nEjemplo — primera Observation:')
mostrar_fhir(observations[0], f'Observation/{observations[0].id}')

#### Ejercicio 3.4 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/observation.html** y responde:

**a)** Además de `laboratory`, ¿qué otras categorías acepta el campo `category`? ¿Cuándo usarías `vital-signs`?

**b)** Usamos `valueQuantity` para el valor numérico. ¿Qué tipo de `value[x]` usarías para un resultado de cultivo de sangre que reporta `'Staphylococcus aureus MRSA'`?

**c)** ¿Qué campo usarías para indicar que el valor de Creatinine (2.5 mg/dL) está fuera del rango de referencia normal (0.6–1.2 mg/dL)?  
(Busca `referenceRange` en la spec)


**Tus respuestas:**

a) ...

b) ...

c) ...


---

## Parte 4 — El Bundle: el sobre del intercambio

**Especificación oficial:** https://hl7.org/fhir/R5/bundle.html

Un `Bundle` agrupa múltiples recursos FHIR para enviarlos juntos.  
Es lo que realmente se intercambia entre sistemas: no recursos sueltos, sino paquetes coherentes.

Las referencias entre recursos forman este grafo:

```
Bundle (type: collection)
│
├── Patient/2
│
├── Encounter/3
│     subject ──────────────────────────────▶ Patient/2
│
├── Condition/4
│     subject ──────────────────────────────▶ Patient/2
│     encounter ────────────────────────────▶ Encounter/3
│
├── Condition/5  (igual)
│
├── Observation/9
│     subject ──────────────────────────────▶ Patient/2
│     encounter ────────────────────────────▶ Encounter/3
│
└── Observation/10 ... (igual para cada labevent)
```


In [ ]:
entries = []


def add_entry(resource):
    entries.append({'resource': json.loads(resource.model_dump_json(exclude_none=True))})


add_entry(patient)
add_entry(encounter)
for c in conditions:
    add_entry(c)
for o in observations:
    add_entry(o)

bundle = Bundle.model_validate({
    'resourceType': 'Bundle',
    'type': 'collection',
    'entry': entries
})

print('Bundle creado:')
print(f'  1  Patient')
print(f'  1  Encounter')
print(f'  {len(conditions)}  Conditions')
print(f'  {len(observations)}  Observations')
print(f'  ─────────────')
print(f'  {len(entries)}  entradas en total')

In [ ]:
bundle_json = bundle.model_dump_json(indent=2, exclude_none=True)
print(bundle_json[:3000])
print('\n[... el JSON continúa ...]')

In [ ]:
out_path = Path(f'bundle_hadm_{HADM_ID}.json')
out_path.write_text(bundle_json, encoding='utf-8')
print(f'Bundle guardado en: {out_path}')
print(f'Tamaño: {len(bundle_json):,} caracteres, {len(entries)} recursos')

#### Ejercicio 4.1 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/bundle.html** y responde:

**a)** ¿Qué diferencia hay entre Bundle `type: collection` y `type: transaction`?  
¿Qué campo adicional tendría cada `entry` en un Bundle de tipo `transaction`?

**b)** ¿Cómo resuelve un servidor FHIR una referencia relativa como `Patient/2` dentro de un Bundle?  
¿Qué pasa si el servidor receptor no tiene ese Patient en su base de datos?


**Tus respuestas:**

a) ...

b) ...


---

## Parte 5 — Ejercicios

Incluye todas las respuestas en el notebook antes de abrir tu Pull Request.


### Ejercicio A — Bundle para otra admisión

Genera el Bundle completo para una admisión **diferente** a `hadm_id = 3`.

**Instrucciones:**

1. Elige una `hadm_id` de la tabla de la Sección 3
2. Cambia el valor de `HADM_ID` en la celda correspondiente
3. Re-ejecuta todas las celdas de la Parte 3 y Parte 4
4. El Bundle se guardará automáticamente como `bundle_hadm_X.json`

**Verifica que:**

- Todas las Conditions referencian `Patient/{id}` y `Encounter/{id}` correctos
- Las Observations con código LOINC tienen `code.coding[0].code` correcto
- El archivo JSON es válido (ejecuta la celda de verificación de abajo)


In [ ]:
# Verificación del Bundle generado
try:
    parsed = json.loads(out_path.read_text(encoding='utf-8'))
    tipos = {}
    for e in parsed['entry']:
        rt = e['resource']['resourceType']
        tipos[rt] = tipos.get(rt, 0) + 1
    print(f'Bundle {out_path.name} — válido')
    for k, v in tipos.items():
        print(f'  {v}  {k}')
except Exception as ex:
    print(f'Error: {ex}')

---

### Ejercicio B — Diagnóstico sin código: implicaciones

En la realidad, siempre hay diagnósticos sin mapeo. Vamos a simular ese caso.

**Instrucciones:**

1. Ejecuta la celda de abajo para insertar un diagnóstico nuevo **sin** código
2. Re-ejecuta la celda de `005_apply_mappings.sql` (es idempotente — no daña los datos existentes)
3. Re-ejecuta la Sección 3.3 (Conditions) y observa la Condition nueva
4. Responde las preguntas


In [ ]:
# Insertar diagnóstico nuevo SIN cobertura en el terminology server
with engine.begin() as conn:
    conn.execute(text('''
        INSERT INTO diagnoses (hadm_id, diagnosis_text)
        VALUES (:hadm_id, 'COVID-19 pneumonia')
        ON CONFLICT DO NOTHING;
    '''), {'hadm_id': HADM_ID})
print('Diagnóstico insertado.')

In [ ]:
# Re-aplicar mapeos (no afecta los ya codificados)
ejecutar_sql('005_apply_mappings.sql')

print('=== Diagnósticos actualizados ===')
display(pd.read_sql('''
    SELECT diagnosis_id, diagnosis_text, code, code_display
    FROM diagnoses
    WHERE hadm_id = %(hadm_id)s
    ORDER BY diagnosis_id;
''', engine, params={'hadm_id': HADM_ID}))

**Preguntas del Ejercicio B:**

**a)** Después de re-ejecutar la Sección 3.3, ¿cómo se ve la Condition de `'COVID-19 pneumonia'` en el JSON? ¿Tiene `code.coding[]` con datos? ¿Qué tiene en su lugar?

**b)** Si un sistema receptor busca Conditions con `code.coding[].code = 'U07.1'` (el código ICD-10 de COVID-19), ¿encontraría esta Condition? ¿Por qué?

**c)** ¿Cuál sería la solución correcta en un entorno de producción? Menciona al menos dos pasos del proceso.


**Tus respuestas:**

a) ...

b) ...

c) ...


---

### Ejercicio C — Bundle filtrado para registro epidemiológico

Un registro nacional solicita un Bundle con formato reducido:

- **Patient**: solo `id`, `gender` y `birthDate` (sin nombre — privacidad)
- **Encounter**: solo el período de hospitalización
- **Observations**: únicamente WBC (LOINC `6690-2`) y Lactate (LOINC `2524-7`)
- **Sin Conditions**: los diagnósticos son confidenciales para este registro

Completa el código y guarda el resultado como `bundle_registry_hadm_{HADM_ID}.json`.


In [ ]:
TARGET_LOINC = {'6690-2', '2524-7'}  # WBC y Lactate

# Patient anonimizado: sin nombre, sin MRN
patient_anon = Patient.model_validate({
    'resourceType': 'Patient',
    'id': str(row['subject_id']),
    # TODO: agrega solo gender y birthDate (sin name ni identifier)
})


def get_loinc_codes(obs):
    """Extrae los códigos LOINC de una Observation."""
    if obs.code.coding is None:
        return set()
    return {c.code for c in obs.code.coding if c.code}


registry_obs = [
    o for o in observations
    if get_loinc_codes(o) & TARGET_LOINC
]

entries_reg = []


def add_reg(res):
    entries_reg.append({'resource': json.loads(res.model_dump_json(exclude_none=True))})


add_reg(patient_anon)
add_reg(encounter)
for o in registry_obs:
    add_reg(o)

bundle_registry = Bundle.model_validate({
    'resourceType': 'Bundle',
    'type': 'collection',
    'entry': entries_reg
})

registry_json = bundle_registry.model_dump_json(indent=2, exclude_none=True)
reg_path = Path(f'bundle_registry_hadm_{HADM_ID}.json')
reg_path.write_text(registry_json, encoding='utf-8')

print(f'Bundle de registro guardado: {reg_path}')
print(f'  {len(entries_reg)} entradas ({len(registry_obs)} Observations)')

---

### Ejercicio D — Reflexión final

Responde en 2–4 frases cada pregunta:

**1.** ¿Qué parte del Lab 2 (terminología) fue indispensable para que los recursos FHIR tuvieran información semántica completa? ¿Qué hubiera pasado si los diagnósticos no tuvieran código ICD-10?

**2.** ¿Qué gana un hospital receptor al recibir un Bundle FHIR en lugar de un CSV o un dump SQL de las mismas tablas?

**3.** El Bundle que generaste es `type: collection`. Para enviarlo a un servidor FHIR real (como HAPI FHIR o Azure Health Data Services), ¿qué debería cambiar?  
(Busca `FHIR transaction bundle` y `fullUrl` en la spec)

**4.** ¿Qué faltaría para que este sistema sea interoperable en producción real? Menciona al menos 3 aspectos (terminología completa, seguridad, perfiles FHIR, validación, API RESTful, etc.)


**Tus respuestas:**

1. ...

2. ...

3. ...

4. ...
